# Binary Tree exercises

- 이산수학에서 그래프 다루면서 트리에 대해 다루긴 했으므로 프로그래밍을 해보지는 않았어도 정의는 알고 있어야 한다
- 데이터구조 과목에서도 자세히 다루고 또 관련 내용을 공부해 본 사람들은 사람들은 프로그램으로도 처리할 줄 알 것이다

이진트리(Tree) 귀납적(재귀적) 정의를 복습
- 어떤 노드(Node)에
    - 데이타(v)가 하나 있고
    - 왼쪽 하위트리(left)도 이진트리이고
    - 오른쪽 하위트리(right)도 이진트리이면
    
  이 노드를 포함한 트리는 이진트리가 된다
- 아무것도 없는 상태(null 혹은 Null)도 이진트리다

```
         2
       /   \
     3      1
   /  \    / \
  .    .   .  .
```

```
         2
       /   \
     3      .
   /  \  
  .    .
```

---
## (버전0): `null`을 이용한 이진트리 정의

`Node`가 곧 이진트리의 정의나 마찬가지

예외적으로, `null`이 될 수 있는 가능성도 있으므로 이진트라는 것은 곧 `Node?` 타입이다

이진트리의 타입을 따로 정의할 필요가 없어서 일단 시작은 간단

(좀더 코드 가독성을 높이려면 typealias를 사용하면 좋음)


트리에 포함된 데이터의 개수를 계산하는 size 메소드를 하나 정의해 보자

In [1]:
data class Node0(val v: Int, val left: Node0?, val right: Node0?) {
    fun size() : Int {
        // 1 + left의 데이터 개수 + right의 데이터 개수
        // null인지 아닌지 항상 신경쓰면서 검사를 해야
        var lSize = 0
        var rSize = 0
        if (left != null)  lSize = left.size()  // 여기에서는 왜 ?.가 아닌 .을 써도 될까?
        if (right != null) rSize = right.size() // 이것에 대해서는 smart cast 관련해 교재를 찾아보세요
        return 1 + lSize + rSize
    }
}

typealias Tree0 = Node0? // C에서 typedef와 비슷 (typealias 정의가 재귀적으로 돌고 돌면 안됨!)
// data class Node0(val v: Int, val left: Tree0, val right: Tree0)

In [2]:
/*
         2
       /   \
     3      .
   /  \  
  .    .

*/

val t0 : Tree0 = Node0(2, Node0(3, null, null), null)

t0

Node0(v=2, left=Node0(v=3, left=null, right=null), right=null)

In [3]:
t0.size()

2

----
## (버전1): 널 오브젝트(null object) 패턴을 이용한 이진트리 정의

참고로 null object 패턴과는 별개로 이진트리보다 더 일반적인 트리 구조(대표적인 예로 파일 시스템의 디렉토리/파일 구조라던가)를 객체지향 용어로는 composite 패턴이라고 부른다. 


`null`을 대신해서 쓸 수 있는 오브젝트라는 뜻에서 널오브젝트 패턴이라 부름

In [4]:
abstract class Tree1() // Tree1을 그냥 만들진 못하고 Node1이나 Null1으로 생성해야 하므로 추상 클래스
data class Node1(val v: Int, val left: Tree1, val right: Tree1) : Tree1()
class Null1() : Tree1() // 아무 데이터도 포함하지 않으면 data class로 하고싶어도 정의 못함

In [5]:
val null1 = Null1()

val t1 : Tree1 = Node1(2, Node1(3, null1, null1), null1)

t1 // Null1의 toString을 오버라이딩하면 좀더 간단하게 출력 가능한데 굳이 여기서 그러지는 않겠음

Node1(v=2, left=Node1(v=3, left=Line_5$Null1@32f232a5, right=Line_5$Null1@32f232a5), right=Line_5$Null1@32f232a5)

In [6]:
val nu1 = Null1()
val nu2 = Null1()

nu1 === nu2 // 매번 다른 오브젝트가 생성됨

false

----
## (버전2): 널 오브젝트를 싱글턴으로 하고 `sealed` 클래스도 활용

아무 데이터도 포함하지 않는 데이터 클래스에 해당하는 것은 싱글턴 오브젝트가 되는 것이 맞다.

그래서 코틀린의 `object` 키워드 활용

일반적은 코틀린 소스코드 파일이라면 sealed 클래스와 하위 클래스를 같은 레벨에서 정의해도 되는데
노트북 환경에서는 sealed 클래스의 하위 클래스를 sealed 클래스 내부에 정의해야만 유의미한 차이를 볼 수 있다

```kotlin
sealed abstract class Tree2() { // sealed가 아닐 때와의 차이점에 대해서 교재 참고
    abstract fun size() : Int // 추상 메소드
}

data class Node2(val v: Int, val left: Tree2, val right: Tree2) : Tree2() {
    override fun size() = 1 + left.size() + right.size()
}

object Null2 : Tree2() {
    override fun toString() = "Null2"
    override fun size() = 0
}
```

sealed 클래스는 하위 클래스가 같은 파일 또는 내부 클래스로 정의된 것으로 한정됨.
그러니까 다른 곳에서 새로운 하위 클래스를 추가로 정의하는 것을 막는다.

In [7]:
sealed abstract class Tree2() { // sealed가 아닐때와 차이점에 대해서 교재 참고
    abstract fun size() : Int // 추상 메소드

    data class Node2(val v: Int, val left: Tree2, val right: Tree2) : Tree2() {
        override fun size() = 1 + left.size() + right.size()
    }
    
    object Null2 : Tree2() {
        override fun toString() = "Null2"
        override fun size() = 0
    }
}

In [8]:
val t2 : Tree2 = Tree2.Node2(2, Tree2.Node2(3, Tree2.Null2, Tree2.Null2), Tree2.Null2)

t2

Node2(v=2, left=Node2(v=3, left=Null2, right=Null2), right=Null2)

In [9]:
t2.size()

2

In [10]:
// java에서 switch 문과 비슷한 기능
val n: Int = 5

when(n) {
    1 -> "one"
    2 -> "two"
    3 -> "three"
    else -> "many" // 다른 나머지 경우
}

many

In [11]:
val t3: Tree2 = Tree2.Null2

when(t3) {
    is Tree2.Null2 -> "Null2" // Tree2를 sealad 클래스로 정의한 경우에는 
    is Tree2.Node2 -> "Node2" // 두가지 경우만으로 모든 경우를 다 고려한 것으로 판단된다
    // else -> "something else" // sealed가 아니면 혹시 다른 곳에서 Tree2를 상속한 하위 클래스가 있을지도 모르니까
}

Null2